# 게시판
### COMMUNITY 
-라이프 : #menuLink1    
-패션 : #menuLink145  
-향수 : #menuLink180  
-코스메틱 : #menuLink188  
-질문 : #menuLink150  
-유머 :  
-OOTD :  
-영화/전시 :   
-여행/맛집 : 
-우수후기 :  
-쇼핑후기 : 

### FASHION


In [1]:
#pip3 install selenium
#pip3 install pandas
#pip3 install requests

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pandas as pd
import pyperclip
import requests
import time
import sys, os
import re
import numpy as np
import warnings

keys = Keys()
options = Options()
options.add_argument('incognito')
warnings.filterwarnings(action='ignore') 

In [6]:
def open_chrome():
    #네이버 로그인 화면 이동 
    url = "https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com"
    
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)
    sleep(20)
    
    return driver

def move_site(driver,cafe_url):
    driver = driver 
    driver.get(cafe_url)
    sleep(5)
    
    return driver  

def move_board(driver, number):
    driver = driver 
    css_selector = f"#menuLink{number}"
    driver.find_element(By.CSS_SELECTOR,css_selector).click()
    sleep(5)
    
    return driver

def move_page(page):
     driver.get(f"https://cafe.naver.com/ArticleSearchList.nhn?search.clubid=27877258&search.media=0&search.searchdate=all&userDisplay=15&search.option=0&search.sortBy=date&search.searchBy=1&search.query=%B4%D9%C0%CC%BF%A1%B1%D7%B3%CE+%BA%ED%B7%E7%C1%BE&search.viewtype=title&search.page={page}")

        
def check_article():
    driver.switch_to.frame("cafe_main")
    n_article = driver.find_elements(By.CLASS_NAME,'nodata')
    driver.switch_to.default_content()
    
    return len(n_article)

def article_n():
    driver.switch_to.frame("cafe_main")
    article_n = len(driver.find_elements(By.CLASS_NAME,"inner_number"))
    driver.switch_to.default_content()
    
    return article_n

def reply_extract(cafename,dataframe):
    d = dataframe
    s = cafename
    
    driver.switch_to.frame("cafe_main")
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
        
    #게시글
    title = soup.select('h3.title_text')[0].text.strip()
    
    #조회수
    views = soup.select('span.count')[0].text.replace('조회', '')
    
    #작성일
    date = soup.select('span.date')[0].text
    
    #url
    cont_url = driver.find_element(By.XPATH,'//*[@id="spiButton"]').get_attribute('data-url')
    
    for i in range(1,len(soup.select('span.text_comment'))+1):
        df = []
        df.append(s)
        df.append(title)
        df.append(cont_url)
        df.append(views)
        df.append(date[:10])
        df.append(soup.select('span.text_comment')[i-1].text)
        d = d.append(pd.Series(df,index=d.columns),ignore_index=True)
    return d

def move_article_page(article_n):
    n = article_n
    driver.switch_to.frame("cafe_main")
    driver.find_element(By.CSS_SELECTOR,f"#main-area > div:nth-child(5) > table > tbody > tr:nth-child({n}) > td.td_article > div.board-list > div > a.article").click()
    driver.switch_to.default_content()
    sleep(2)

def back_page():
    driver.back()
    driver.switch_to.default_content()
    sleep(2)

In [3]:
cafe_url = "https://cafe.naver.com/coredenim"
board_number = 145

In [7]:
#driver = open_chrome()
#driver = move_site(driver, cafe_url)
#driver = move_board(driver, board_number)

In [5]:
css_selector = "#menuLink145"
print(css_selector)    
#driver.switch_to.frame("cafe_main")
driver.find_element(By.CSS_SELECTOR,css_selector).click()
   # driver.switch_to.default_content()
   # sleep(5)
    
    #return driver

#menuLink145


In [ ]:
#site url
url = "https://cafe.naver.com/coredenim"
cafename = "브랜디드"
driver.get(url)

In [ ]:
d = pd.DataFrame(columns=["카페명","게시물","url","조회수","작성날짜","댓글"])
d3 = pd.DataFrame(columns=["카페명","게시물","url","조회수","작성날짜","댓글"])

In [ ]:
txt = "다이에그널 블루종"
element = driver.find_element(By.ID,'topLayerQueryInput') 
element.send_keys(txt)              
element.submit()   

search = driver.find_element(By.CSS_SELECTOR,'#cafe-search > form > button')
search.click()

driver.switch_to.frame("cafe_main")
search = driver.find_element(By.CSS_SELECTOR,'#currentSearchByTop')
search.click()
search = driver.find_element(By.XPATH,'html/body/div[1]/div/div[1]/div[1]/form/div[3]/ul[1]/li[2]/a')
search.click()
search = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div[1]/form/div[4]/button')
search.click()

In [ ]:
page = 0
while page > -1:
    try:
        page = page+1
        print(f"현재 {page}번째 페이지입니다")
        
        #페이지 이동
        print("페이지 이동")
        move_page(page)
        sleep(5)
        
        #게시물 확인
        if check_article() > 0:
            print("게시물이 확인되지 않습니다.")
            break
        else:
            pass
        
        #게시물 갯수 확인
        n = article_n()
        print(f"게시물 갯수는 {n}개 입니다")
        
        for i in range(1,n):
            print(f"{i}번째 게시글로 이동")
            move_article_page(i)
            d2 = reply_extract(cafename,d)
            d3 = d3.append(d2)
            sleep(5)
            print("뒤로가기")
            back_page()
            
    except:
        pass

In [ ]:
#d3 = d3.reset_index()
#d3 = d3.drop(["level_0","index"],axis=1)
d3.to_csv('/Users/jisoo.kim/Downloads/navercafe.csv',encoding='utf-8')